## 11.3 Constraint satisfaction

Sometimes we must search for each item that satisfies the conditions.
Other times we must search for multiple items that _together_ satisfy the conditions.
That's a form of **constraint satisfaction problem** (**CSP**). The earlier
example of checking if a password is valid is a CSP with two constraints:
the string must include a lowercase letter and a digit.
We search for two candidates (characters) that together
satisfy the conditions because no character can satisfy both by itself.

CSPs occur often in business, engineering, manufacturing and many other domains.
A classic CSP is timetabling: which class should be taught where and when?
Constraints include teaching each class in a sufficiently large room,
teaching some classes in specialised rooms (e.g. labs), and making sure
no one is scheduled for different classes at the same time.

Constraints are often described with mathematical equations or inequalities.
Solving a CSP amounts to assigning values to all variables so that the
constraints are satisfied.
There are advanced specialised techniques to solve CSPs.
In M269 we'll solve them with exhaustive search only.

This section introduces more techniques to make exhaustive search faster.

### 11.3.1 Problem

Consider the following CSP.

> Given positive integers _sum_, _product_ and _square sum_,
> find all distinct integers _x_, _y_ and _z_ such that
> _x_+_y_+_z_ = _sum_, _x_×_y_×_z_ = _product_ and
> *x*² + *y*² + *z*² = _square sum_.

This problem has three constraints in the form of equations
and another three constraints in the form of inequalities:
_x_ ≠ _y_, _y_ ≠ _z_ and _z_ ≠ _x_.

Some examples:

_sum_ | _product_ | _square sum_ | _x_ | _y_ | _z_
-|-|-|-|-|-
6 | 6 | 14 | 1 | 2 | 3
0 | 6 | 14 | −1 | −2 | 3
21 | 336  | 149 | 6 | 7 | 8
33 | 1320 | 365 | 10 | 11 | 12

<div class="alert alert-info">
<strong>Info:</strong> This is Online Judge problem
<a href="https://uva.onlinejudge.org/external/115/11565.pdf">Simple Equations</a>.
</div>

### 11.3.2 Algorithm and complexity

The solutions are triples of integers, so
the output will be a set of tuples (_x_, _y_, _z_).
We generate all possible triples with three nested loops.
First we must determine a range of possible candidates for each integer.

The sum doesn't constrain the values: I can set _x_ as small or large as I want
and still find a solution (_y_ = –_x_ and _z_ = _sum_).
However, the product equation forces each value to be in the range
from –_product_ to _product_. If any value is outside that range, then
one of the other two is a real number between −1 and 1, not an integer.

1. let _solutions_ be the empty set
1. for each _x_ from –_product_ to _product_:
   1. for each _y_ from –_product_ to _product_:
      1. for each _z_ from –_product_ to _product_:
         1. if _x_, _y_ and _z_ satisfy the constraints:
            1. add (_x_, _y_, _z_) to _solutions_

Steps 2, 2.1 and 2.1.1 generate the candidates and step&nbsp;2.1.1.1 tests them:
it checks _x_ ≠ _y_ ≠ _z_ ≠ _x_ and the three equations.
A single Boolean expression for checking four constraints is too long,
so I'll implement the test with a separate auxiliary function.

Each for-loop does 2 × _product_ iterations.
Testing a candidate requires a fixed number of arithmetic operations.
The overall complexity is:

(2 × _product_)³ × Θ(1) = 8 × *product*³ × Θ(1) = Θ(*product*³).

This is called **cubic** complexity.
Quadratic algorithms are to be avoided when possible;
cubic algorithms even more so.

### 11.3.3 Code and performance

Let's implement the algorithm to see how slowly it runs.

In Python we can nest functions inside each other.
This is useful to write auxiliary functions that aren't used by anyone else.
The inner function can access the arguments of the outer function.

In [1]:
def equations(sum: int, product: int, square_sum: int) -> set:
    """Return all triples that satisfy the constraints.

    Preconditions: sum > 0, product > 0, square_sum > 0
    Postconditions: the output has exactly all (x, y, z) such that
    - x ≠ y ≠ z ≠ x and x, y, z are integers
    - x+y+z = sum, x*y*z = product, x² + y² + z² = square_sum
    """

    def satisfies(x: int, y: int, z: int) -> bool:
        """Check if x, y and z satisfy the constraints."""
        if x == y or y == z or z == x:
            return False
        if x + y + z != sum or x*y*z != product:
            return False
        return x*x + y*y + z*z == square_sum

    solutions = set()
    for x in range(-product, product+1):
        for y in range(-product, product+1):
            for z in range(-product, product+1):
                if satisfies(x, y, z):
                    solutions.add( (x, y, z) )
    return solutions

Let's test this with the simplest example.

In [2]:
equations(6, 6, 14)

{(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)}

Oh dear! We get several times the same solution, only in a different order.
I'll deal with that in a moment. First, let's time the execution.

In [3]:
%timeit -r 3 equations(6, 6, 14)

643 µs ± 61.4 µs per loop (mean ± std. dev. of 3 runs, 1000 loops each)


This example required (2×6)³ = 1728 iterations
(because each loop goes from −6 to 6)
and took on average about 400&nbsp;µs / 1600 = 0.25&nbsp;µs = 250&nbsp;ns
per iteration on my machine.

#### Exercise 11.3.1

It's useful to do back-of-the-envelope estimates of the running time,
to help decide if the performance is acceptable or a better algorithm is needed.

Using Python as a calculator, write an expression that computes the
time, in seconds, for the algorithm to solve the example with _product_ = 336, assuming each iteration takes 250&nbsp;ns.
There are one billion, i.e. a thousand million, nanoseconds in a second.

In [4]:
pass

Now write an expression to compute the time in minutes for solving the example
with _product_ = 1320.

In [5]:
pass

[Hint](../31_Hints/Hints_11_3_01.ipynb)
[Answer](../32_Answers/Answers_11_3_01.ipynb)

<div class="alert alert-warning">
<strong>Note:</strong> Knowing the complexity and the run-time for a small input,
you can estimate the run-times for large inputs.
</div>

### 11.3.4 Don't generate equivalent candidates

The algorithm is generating all
[permutations](../04_Iteration/04_1_sequences.ipynb#4.1.2.3-Sorting) of the same values
because the order in which values are added or multiplied doesn't matter.
All those solutions are equivalent; we should generate only one of them.

The easiest way to generate only one of several permutations is
to sort the values, e.g. to only generate triples with _x_ < _y_ < _z_.
This ensures the three values are distinct by construction, which
makes the test simpler and more efficient,
as it only has to check the equations.

1. let _solutions_ be the empty set
1. for each _x_ from –_product_ to _product_:
   1. for each _y_ from _x_ + 1 to _product_:
      1. for each _z_ from _y_ + 1 to _product_:
         1. if _x_, _y_ and _z_ satisfy the equations:
            1. add (_x_, _y_, _z_) to _solutions_

Imposing an order on the triple's values reduces the size of the output set
and, more importantly, of the search space:
steps 2.1 and 2.1.1 now generate fewer candidates for _y_ and _z_.

### 11.3.5 Reduce the range

Another technique is to avoid generating candidates that will fail the test.
In the case of integers, that means making the range of candidates as small as
possible, while still making sure it contains all solutions.

We only used the product equation to set the initial search space.
The equation

*x*² + *y*² + *z*² = _square sum_

may be useful too.
Square numbers are never negative: if any of them
is larger than _square sum_, then the constraint can't be satisfied.
So _x_, _y_ and _z_ must be in the range
–$\sqrt{\textit{square sum}}$ to $\sqrt{\textit{square sum}}$.

When we have multiple ranges, the values must be in their intersection.
In this case, one range is contained in the other, because both go from
−_limit_ to +_limit_ for some integer _limit_, so we take the smallest range.
The algorithm becomes:

1. let _solutions_ be the empty set
1. let _limit_ be min(_product_, floor(sqrt(_square sum_)))
1. for each _x_ from –_limit_ to _limit_:
   1. for each _y_ from _x_ + 1 to _limit_:
      1. for each _z_ from _y_ + 1 to _limit_:
         1. if _x_, _y_ and _z_ satisfy the equations:
            1. add (_x_, _y_, _z_) to _solutions_

Consider the example with _product_ = 336 and _square sum_ = 149.
The original algorithm generates (2 × 336)³ ≈ 303 million candidates;
the new algorithm generates far fewer than
(2 × floor($\sqrt{149}$))³ = 24³ ≈ 14 thousand, because more often
than not the loops for _y_ and _z_ don't start at −149.
A vast reduction in the search space!

### 11.3.6 Compute part of a candidate

Last but not least,
when a constraint creates a dependency between values, we can directly
compute one value from the others instead of searching for it.

<div class="alert alert-warning">
<strong>Note:</strong> The best way to make searches efficient is to avoid searches.
</div>

For this problem,
once we have candidate values for _x_ and _y_ we can use one equation to
determine the value of _z_ and then check the other two equations.
The sum equation is the simplest to compute _z_.

1. let _solutions_ be the empty set
1. let _limit_ be min(_product_, floor(sqrt(_square sum_)))
1. for each _x_ from –_limit_ to _limit_:
   1. for each _y_ from _x_ + 1 to _limit_:
      1. let _z_ be _sum_ − _x_ − _y_
      1. if _y_ < _z_ and _x_×_y_×_z_ = _product_ and *x*²+*y*²+*z*² = _square sum_:
         1. add (_x_, _y_, _z_) to _solutions_

Since _z_ is no longer generated by a loop starting at _y_ + 1,
we must explicitly check for _y_ < _z_.

Continuing with the same example, the new algorithm generates at
most 24² = 576 candidates, down from 24³ ≈ 14 thousand.

### 11.3.7 Improved code and performance

Let's implement the final algorithm, test it and
measure its performance. I won't repeat the docstring.

In [6]:
import math

def fast_equations(sum: int, product: int, square_sum: int) -> set:
    solutions = set()
    limit = min(product, math.floor(math.sqrt(square_sum)))
    for x in range(-limit, limit + 1):
        for y in range(x+1, limit + 1):
            z = sum - x - y
            if y < z and x*y*z == product and x*x+y*y+z*z == square_sum:
                solutions.add( (x, y, z) )
    return solutions

In [7]:
print(fast_equations(6, 6, 14))
%timeit -r 3 fast_equations(6, 6, 14)

{(1, 2, 3)}
5.87 µs ± 443 ns per loop (mean ± std. dev. of 3 runs, 100000 loops each)


In [8]:
print(fast_equations(21, 336, 149))
%timeit -r 3 fast_equations(21, 336, 149)

{(6, 7, 8)}
53.2 µs ± 4.31 µs per loop (mean ± std. dev. of 3 runs, 10000 loops each)


In [9]:
print(fast_equations(33, 1320, 365))
%timeit -r 3 fast_equations(33, 1320, 365)

{(10, 11, 12)}
128 µs ± 4.49 µs per loop (mean ± std. dev. of 3 runs, 10000 loops each)


The complexity has gone down from Θ(*product*³) to\
Θ(min(_product_, $\sqrt{\textit{square sum}}$)²) =
Θ(min(*product*², _square sum_)).

An initially cubic algorithm has become quadratic in one input
or linear in another,
whatever happens to be best for the particular input values.
This makes a substantial difference.
An expected run-time of one-and-a-quarter hours shrinks to a few microseconds.
That's the power of systematically thinking about the constraints
and using them to reduce the search space.

⟵ [Previous section](11_2_factorisation.ipynb) | [Up](11-introduction.ipynb) | [Next section](11_4_permutations.ipynb) ⟶